# ***Necessidades do projeto***

In [17]:
#Necessidades.
# No terminal, use: pip install requests
# No terminal, use: pip install pandas
# No terminal, use: pip install plyer
# No terminal, use: pip install sqlite3



import requests
import pandas as pd
from plyer import notification
from datetime import datetime
import sqlite3

# ***Criando as funções***

In [11]:
# Notificação
def alerta(nivel, base, etapa, sucesso=False):
    if sucesso:
        titulo = "Sucesso"
        mensagem = f"Extração realizada com sucesso para a base {base} na etapa {etapa}."
    else:
        if nivel == 1:
            titulo = "Alerta Baixo"
        elif nivel == 2:
            titulo = "Alerta Médio"
        elif nivel == 3:
            titulo = "Alerta Alto"
        else:
            titulo = "Alerta Desconhecido"
        
        mensagem = f"Falha no carregamento da base {base} na etapa {etapa};"
    
    data_atual = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    notification.notify(
        title=f"ATENÇÃO: {titulo}",
        message=f"{mensagem}\n{data_atual}",
        app_name='API_NoobMaster',
        timeout=10
    )

# URLs das APIs
urls = {
    "Europa": "https://restcountries.com/v3.1/region/europe",
    "América": "https://restcountries.com/v3.1/region/america",
    "Africa": "https://restcountries.com/v3.1/region/africa"
}

# Extração dos dados
def extrair_dados(urls):
    tabelas = []  # Para armazenar as tabelas extraídas
    for idioma, url in urls.items():
        try:
            response = requests.get(url)
            response.raise_for_status()
            dados = response.json()
            print(f"Extração bem-sucedida da base {idioma}")
            
            # Notificação de sucesso
            alerta(nivel=1, base=idioma, etapa="EXTRACAO", sucesso=True)
            
            # Transformar os dados em um DataFrame
            df = pd.json_normalize(dados)
            tabelas.append(df)
            
        except requests.exceptions.RequestException as e:
            # Caso ocorra algum erro, gerar alerta
            alerta(nivel=2, base=idioma, etapa="EXTRACAO")
            print(f"Erro na extração da base {idioma}: {e}")

    return tabelas

# Tratamento dos dados
def tratar_dados(tabelas):
    tabelas_tratadas = []  # Lista para armazenar as tabelas tratadas
    for idx, tabela in enumerate(tabelas):
        
        # Colunas que EU QUERO!!!
        colunas_interesse = ['name.common', 'name.official', 'capital', 'region', 'subregion', 'population', 'area', 'flag', 'timezones', 'continents', 'demonyms.eng.m']
        tabela_tratada = tabela[colunas_interesse].copy()
        tabela_tratada['capital'] = tabela_tratada['capital'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)

        tabela_tratada = tabela_tratada.rename(columns={
            'name.common': 'Nome Comum',
            'name.official': 'Nome Oficial',
            'capital': 'Capital',
            'region': 'Região',
            'subregion': 'Sub Região',
            'population': 'População',
            'area': 'Área',
            'flag': 'Sigla do País',
            'timezones': 'Fuso horário',
            'continents': 'Continente',
            'demonyms.eng.m': 'Nome População'
        })
        
        tabelas_tratadas.append(tabela_tratada)
       

    return tabelas_tratadas



# ***Chamando as funções***

In [12]:
tabelas_extraidas = extrair_dados(urls)
tabelas_tratadas = tratar_dados(tabelas_extraidas)

Extração bem-sucedida da base Europa
Extração bem-sucedida da base América
Extração bem-sucedida da base Africa


# ***Exibir primeira tabela***

In [13]:
tabela_1 = tabelas_tratadas[0]
print("\nTabela 1:")
tabela_1


Tabela 1:


,Nome Comum,Nome Oficial,Capital,Região,Sub Região,População,Área,Sigla do País,Fuso horário,Continente,Nome População
0,Norway,Kingdom of Norway,Oslo,Europe,Northern Europe,5379475,323802.00,🇳🇴,[UTC+01:00],[Europe],Norwegian
1,Greece,Hellenic Republic,Athens,Europe,Southern Europe,10715549,131990.00,🇬🇷,[UTC+02:00],[Europe],Greek
2,Åland Islands,Åland Islands,Mariehamn,Europe,Northern Europe,29458,1580.00,🇦🇽,[UTC+02:00],[Europe],Ålandish
3,Switzerland,Swiss Confederation,Bern,Europe,Western Europe,8654622,41284.00,🇨🇭,[UTC+01:00],[Europe],Swiss
4,Croatia,Republic of Croatia,Zagreb,Europe,Southeast Europe,4047200,56594.00,🇭🇷,[UTC+01:00],[Europe],Croatian
5,Iceland,Iceland,Reykjavik,Europe,Northern Europe,366425,103000.00,🇮🇸,[UTC],[Europe],Icelander
6,Luxembourg,Grand Duchy of Luxembourg,Luxembourg,Europe,Western Europe,632275,2586.00,🇱🇺,[UTC+01:00],[Europe],Luxembourger
7,Hungary,Hungary,Budapest,Europe,Central Europe,9749763,93028.00,🇭🇺,[UTC+01:00],[Europe],Hungarian
8,Netherlands,Kingdom of the Netherlands,Amsterdam,Europe,Western Europe,16655799,41850.00,🇳🇱,[UTC+01:00],[Europe],Dutch
9,Lithuania,Republic of Lithuania,Vilnius,Europe,Northern Europe,2794700,65300.00,🇱🇹,[UTC+02:00],[Europe],Lithuanian


# ***Exibir segunda tabela***

In [14]:
tabela_2 = tabelas_tratadas[1]
print("\nTabela 2:")
tabela_2


Tabela 2:


,Nome Comum,Nome Oficial,Capital,Região,Sub Região,População,Área,Sigla do País,Fuso horário,Continente,Nome População
0,Colombia,Republic of Colombia,Bogotá,Americas,South America,50882884,1141748.0,🇨🇴,[UTC-05:00],[South America],Colombian
1,Curaçao,Country of Curaçao,Willemstad,Americas,Caribbean,155014,444.0,🇨🇼,[UTC-04:00],[North America],Curaçaoan
2,French Guiana,Guiana,Cayenne,Americas,South America,254541,83534.0,🇬🇫,[UTC-03:00],[South America],Guianan
3,Grenada,Grenada,St. George's,Americas,Caribbean,112519,344.0,🇬🇩,[UTC-04:00],[North America],Grenadian
4,Sint Maarten,Sint Maarten,Philipsburg,Americas,Caribbean,40812,34.0,🇸🇽,[UTC-04:00],[North America],St. Maartener
5,Argentina,Argentine Republic,Buenos Aires,Americas,South America,45376763,2780400.0,🇦🇷,[UTC-03:00],[South America],Argentine
6,Turks and Caicos Islands,Turks and Caicos Islands,Cockburn Town,Americas,Caribbean,38718,948.0,🇹🇨,[UTC-04:00],[North America],Turks and Caicos Islander
7,Dominica,Commonwealth of Dominica,Roseau,Americas,Caribbean,71991,751.0,🇩🇲,[UTC-04:00],[North America],Dominican
8,United States Minor Outlying Islands,United States Minor Outlying Islands,Washington DC,Americas,North America,300,34.2,🇺🇲,"[UTC-11:00, UTC-10:00, UTC+12:00]",[Oceania],American Islander
9,Costa Rica,Republic of Costa Rica,San José,Americas,Central America,5094114,51100.0,🇨🇷,[UTC-06:00],[North America],Costa Rican


# ***Exibir terceira tabela***

In [15]:
tabela_3 = tabelas_tratadas[2]
print("\nTabela 3:")
tabela_3


Tabela 3:


,Nome Comum,Nome Oficial,Capital,Região,Sub Região,População,Área,Sigla do País,Fuso horário,Continente,Nome População
0,Lesotho,Kingdom of Lesotho,Maseru,Africa,Southern Africa,2142252,30355.0,🇱🇸,[UTC+02:00],[Africa],Mosotho
1,Central African Republic,Central African Republic,Bangui,Africa,Middle Africa,4829764,622984.0,🇨🇫,[UTC+01:00],[Africa],Central African
2,Morocco,Kingdom of Morocco,Rabat,Africa,Northern Africa,36910558,446550.0,🇲🇦,[UTC],[Africa],Moroccan
3,Sierra Leone,Republic of Sierra Leone,Freetown,Africa,Western Africa,7976985,71740.0,🇸🇱,[UTC],[Africa],Sierra Leonean
4,Burkina Faso,Burkina Faso,Ouagadougou,Africa,Western Africa,20903278,272967.0,🇧🇫,[UTC],[Africa],Burkinabe
5,Eritrea,State of Eritrea,Asmara,Africa,Eastern Africa,5352000,117600.0,🇪🇷,[UTC+03:00],[Africa],Eritrean
6,Tanzania,United Republic of Tanzania,Dodoma,Africa,Eastern Africa,59734213,945087.0,🇹🇿,[UTC+03:00],[Africa],Tanzanian
7,Algeria,People's Democratic Republic of Algeria,Algiers,Africa,Northern Africa,44700000,2381741.0,🇩🇿,[UTC+01:00],[Africa],Algerian
8,Mauritania,Islamic Republic of Mauritania,Nouakchott,Africa,Western Africa,4649660,1030700.0,🇲🇷,[UTC],[Africa],Mauritanian
9,Sudan,Republic of the Sudan,Khartoum,Africa,Northern Africa,43849269,1886068.0,🇸🇩,[UTC+03:00],[Africa],Sudanese


In [22]:
tabela_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Nome Comum      53 non-null     object 
 1   Nome Oficial    53 non-null     object 
 2   Capital         53 non-null     object 
 3   Região          53 non-null     object 
 4   Sub Região      53 non-null     object 
 5   População       53 non-null     int64  
 6   Área            53 non-null     float64
 7   Sigla do País   53 non-null     object 
 8   Fuso horário    53 non-null     object 
 9   Continente      53 non-null     object 
 10  Nome População  53 non-null     object 
dtypes: float64(1), int64(1), object(9)
memory usage: 4.7+ KB


# ***Banco de dados SQLite***
### ***Converter e adicionar ao banco de dados***

In [23]:
# Função para converter colunas que têm o tipo object em strings
def converter_para_string(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].astype(str)
    return df

tabela_1 = converter_para_string(tabela_1)
tabela_2 = converter_para_string(tabela_2)
tabela_3 = converter_para_string(tabela_3)

# Conexão SQLite
conn = sqlite3.connect('meu_banco_de_dados.db')

# Adicionar as tabelas
tabela_1.to_sql('europa', conn, if_exists='replace', index=False)
tabela_2.to_sql('america', conn, if_exists='replace', index=False)
tabela_3.to_sql('africa', conn, if_exists='replace', index=False)
conn.close()

### ***Consulta ao banco de dados SQLite***

In [35]:
# Conexão ao banco
conn = sqlite3.connect('meu_banco_de_dados.db')

query = "SELECT [Nome Oficial] FROM europa"
df = pd.read_sql(query, conn)
print(df)

# Fechar a conexão
conn.close()


                                         Nome Oficial
0                                   Kingdom of Norway
1                                   Hellenic Republic
2                                       Åland Islands
3                                 Swiss Confederation
4                                 Republic of Croatia
5                                             Iceland
6                           Grand Duchy of Luxembourg
7                                             Hungary
8                          Kingdom of the Netherlands
9                               Republic of Lithuania
10                                    Slovak Republic
11                      Principality of Liechtenstein
12                                Republic of Moldova
13                                   Italian Republic
14                                Bailiwick of Jersey
15                             Principality of Monaco
16                                Republic of Belarus
17                          